In [ ]:
!pip install pandas

In [70]:
## Read data from file to dataframe using pands
import pandas as pd
Review  = pd.read_csv('c:\\Users\\pucha\\Desktop\\Naveen\\generative ai\\IMDBDataset.csv')
type(Review)
Review

,message,label
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [137]:
# Define a method to Remove URLS , HTML Tags and emojis from dataset
import re

def remove_url_htmltags_emojis(text):
    pattern = re.compile(r'https?://\S+|www\.\S+|<.*?>|['
                              u'\U0001F600-\U0001F64F'  # emoticons
                              u'\U0001F300-\U0001F5FF'  # symbols & pictographs
                              u'\U0001F680-\U0001F6FF'  # transport & map symbols
                              u'\U0001F700-\U0001F77F'  # alchemical symbols
                              u'\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
                              u'\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
                              u'\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
                              u'\U0001FA00-\U0001FA6F'  # Chess Symbols
                              u'\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
                              u'\U00002702-\U000027B0'  # Dingbats
                              u'\U000024C2-\U0001F251'
                              ']+', flags=re.UNICODE)
    return pattern.sub("",text)



# Define a method to remove punctuations
import string

def remove_punctuations(text):
    # Define the set of punctuations
    punctuation_set = set(string.punctuation)
    # Remove punctuations from the text
    cleaned_text = ''.join(char for char in text if char not in punctuation_set)
    return cleaned_text

import re

# Define a dictionary of chat words and their replacements
chat_words_dict = {
    "lol": "laugh out loud",
    "brb": "be right back",
    "ttyl": "talk to you later",
    # Add more chat words and their replacements as needed
}

# Define a method to replace chat words with full text

def replace_chatwords(text):
    # Create a regular expression pattern to match chat words
    pattern = re.compile(r'\b(?:%s)\b' % '|'.join(re.escape(word) for word in chat_words_dict.keys()), re.IGNORECASE)
    
    # Replace chat words with their corresponding replacements
    replaced_text = pattern.sub(lambda x: chat_words_dict[x.group().lower()], text)
    
    return replaced_text

# Spelling Correction

from textblob import TextBlob

def correct_spelling(text):
    # Create a TextBlob object
    blob = TextBlob(text)
    # Correct the spelling of the text
    corrected_text = str(blob.correct())
    
    return corrected_text

    

In [170]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm import tqdm
#Create an instance of the porter stemmer
ps=PorterStemmer()
corpus=[]
for i in tqdm(range (0,len(Review["message"]))):
  subject = remove_url_htmltags_emojis(Review["message"][i])
  #subject = correct_spelling(subject)
  #removing all other characters except a-z,A-z
  subject = re.sub('[^a-zA-z]',' ',subject)
  subject = subject.lower()
  subject = subject.split()
  subject = [ps.stem(word) for word in subject if word not in (stopwords.words("english"))]
  subject = ' '.join(subject)
  corpus.append(subject)

100%|██████████| 10000/10000 [10:16<00:00, 16.21it/s]


In [171]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import  TfidfVectorizer
cv = CountVectorizer(max_features=2500)
X=cv.fit_transform(corpus).toarray()

In [172]:
X.shape

(10000, 2500)

In [173]:
Y=pd.get_dummies(Review["label"])
Y=Y.iloc[:,1].values

In [174]:
Y.shape

(10000,)

In [175]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, Y_train)

In [176]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, Y_train)

In [177]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [178]:
from sklearn.metrics import accuracy_score,classification_report

In [179]:
score=accuracy_score(Y_test,y_pred)
print(score)

0.8435


In [180]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,Y_test))

              precision    recall  f1-score   support

       False       0.85      0.84      0.84       990
        True       0.84      0.85      0.85      1010

    accuracy                           0.84      2000
   macro avg       0.84      0.84      0.84      2000
weighted avg       0.84      0.84      0.84      2000



In [181]:
tv = TfidfVectorizer(max_features=2500)
X = tv.fit_transform(corpus).toarray()

In [182]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [183]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, Y_train)

In [184]:
#prediction
y_pred=spam_detect_model.predict(X_test)

In [185]:
score=accuracy_score(Y_test,y_pred)
print(score)

0.8475


In [186]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,Y_test))

              precision    recall  f1-score   support

       False       0.83      0.86      0.84       946
        True       0.87      0.84      0.85      1054

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



Word2vec Implementation

In [112]:
!pip install gensim

In [41]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [216]:
len(Y)

10000

In [218]:
words=[]
nullindex=[]
for i,sent in enumerate(corpus):
    sent_token=sent_tokenize(sent)

    #if you have null list use this code 
    """
    if len(sent_token)==0:
        nullindex.append(i)
        words.append(sent_token)
    else: """
    for sent in sent_token:
        words.append(simple_preprocess(sent))

    

In [245]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  'receivea',
 

In [192]:
# check rows input/output
len(words)
len(Y)

10000

In [252]:
""" Use this code when you use null list index to remove null list values from inputs/outputs for model
words = [item for index, item in enumerate(words) if index not in nullindex]
Y = [item for index, item in enumerate(Y) if index not in nullindex] """

In [256]:
# check rows input/output
len(words)
len(Y)

5564

In [53]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  'receivea',
 

In [195]:
import gensim
### Lets train Word2vec from scratch
model=gensim.models.Word2Vec(words,window=5,min_count=1)

In [196]:
model.corpus_count

10000

In [197]:
model.wv.index_to_key

['movi',
 'film',
 'one',
 'like',
 'time',
 'good',
 'make',
 'get',
 'charact',
 'watch',
 'see',
 'even',
 'would',
 'stori',
 'realli',
 'well',
 'scene',
 'show',
 'look',
 'much',
 'go',
 'bad',
 'great',
 'end',
 'love',
 'peopl',
 'first',
 'act',
 'also',
 'way',
 'play',
 'think',
 'made',
 'thing',
 'know',
 'say',
 'could',
 'work',
 'seem',
 'come',
 'plot',
 'never',
 'actor',
 'year',
 'littl',
 'take',
 'mani',
 'seen',
 'two',
 'want',
 'best',
 'life',
 'tri',
 'ever',
 'better',
 'man',
 'give',
 'still',
 'perform',
 'feel',
 'use',
 'someth',
 'director',
 'actual',
 'find',
 'lot',
 'part',
 'real',
 'back',
 'interest',
 'guy',
 'though',
 'old',
 'music',
 'star',
 'funni',
 'live',
 'cast',
 'noth',
 'set',
 'new',
 'enjoy',
 'point',
 'role',
 'anoth',
 'girl',
 'day',
 'everi',
 'start',
 'origin',
 'turn',
 'direct',
 'pretti',
 'horror',
 'world',
 'kill',
 'believ',
 'us',
 'effect',
 'minut',
 'wonder',
 'quit',
 'comedi',
 'around',
 'thought',
 'long',


In [198]:
model.wv["new"].shape

(100,)

In [199]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    return np.mean([model.wv[wor] for wor in doc if wor in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)
        

In [203]:
import numpy as np
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 10000/10000 [00:38<00:00, 258.80it/s]


In [264]:
"""checking Nan values/ Incorrect Dimension  as we are removing OOV data in input data X

nan_list=[]
for i,j in enumerate(X):
  if j.size != 100:
    nan_list.append(i)

X_new = [item for index, item in enumerate(X) if index not in nan_list]
Y_new = [item for index, item in enumerate(Y) if index not in nan_list]

len(X)
len(Y)
"""
  

In [207]:
X=np.array(X)

In [208]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [277]:
type(X_train)

numpy.ndarray

In [282]:
test = X[X<0]

In [291]:
test.size

253392

In [209]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [210]:
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [211]:
y_pred = rf_classifier.predict(X_test)

In [212]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7875


In [213]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

       False       0.76      0.80      0.78       928
        True       0.82      0.78      0.80      1072

    accuracy                           0.79      2000
   macro avg       0.79      0.79      0.79      2000
weighted avg       0.79      0.79      0.79      2000

